In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Extração de informações de textos com  Generative AI na Vertex AI 

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

A extração de informações é um processo que pode ser feito manualmente ou automaticamente. A extração manual de informações é o processo de ler o documento e copiar informações para um novo documento. A extração automática de informações é o processo de usar um software para extrair informações do documento.

A extração de informações de textos pode ser usada para uma variedade de propósitos. Um propósito comum é converter documentos em um formato legível por máquina. Isso pode ser útil para armazenar documentos em um banco de dados ou para processar documentos com software. Outra finalidade comum é extrair informações de documentos. Isso pode ser útil para localizar informações específicas em um documento ou para resumir o conteúdo de um documento.

Modelos de linguagem grandes (LLMs) são bons para extração de informações porque são treinados em conjuntos de dados massivos de texto e código, o que lhes permite aprender as relações entre palavras e frases. Eles também podem entender o contexto do texto e gerar texto, o que lhes permite extrair informações que não são explicitamente declaradas ou preencher as lacunas no texto que estão faltando. As respostas dos LLMs também podem ser melhoradas por meio de métodos como prompts de poucos disparos.

Saiba mais sobre os prompts de extração na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/extraction-prompts).

### Objetivo

Neste tutorial, você aprenderá como usar modelos generativos para extrair as informações de textos trabalhando com os seguintes exemplos:
- Extração de especificações técnicas do Google Pixel
- Solução de problemas de WiFi
- Responder a perguntas como um personagem
- Convertendo uma lista de ingredientes para o formato JSON
- Organizar os resultados de uma extração de texto

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI e da Cloud Translate API

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

from vertexai.language_models import TextGenerationModel

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Extração de informações de texto

### Extração das especificações técnicas do Google Pixel

Neste exemplo, você irá extrair as especificações técnicas de um telefone Pixel do texto no formato JSON usando a API PaLM.

In [ ]:
prompt = """
Extraia as especificações técnicas do texto abaixo no formato JSON.

Texto: Google Nest WiFi, velocidade de rede de até 1200Mpbs, frequências de 2,4 GHz e 5 GHz, protocolo WP3
JSON: {
   "produto":"Google Nest WiFi",
   "velocidade":"1200Mpbs",
   "frequências": ["2,4GHz", "5GHz"],
   "protocolo":"WP3"
}

Texto: Google Pixel 7, rede 5G, 8 GB de RAM, processador Tensor G2, 128 GB de armazenamento, coral
JSON:
"""

print(
    generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text
)

### Solução de problemas de WiFi

Neste exemplo, você pede ao LLM para responder a uma pergunta sobre a solução de problemas de um roteador Google WiFi com base na descrição das diferentes luzes de status no roteador. O modelo só poderá responder com o texto que foi fornecido, o que ajuda a evitar que gere respostas potencialmente incorretas. Aqui está como você pode fazer isso usando a API PaLM.

In [ ]:
prompt = """
Responda a questão usando o texto abaixo. Responda apenas com o texto fornecido. Responda em português.
Pergunta: O que devo fazer para consertar meu WiFi desconectado? A luz no meu roteador Google WiFi está amarela e piscando lentamente.

Texto:
Cor: Sem luz
O que significa: o roteador não tem energia ou a luz foi reduzida no aplicativo.
O que fazer:
Verifique se o cabo de alimentação está conectado corretamente ao roteador e a uma tomada que esteja funcionando.
Se o seu dispositivo já estiver configurado e a luz estiver apagada, verifique as configurações de brilho da luz no aplicativo.
Se ainda não houver luz, entre em contato com o suporte ao cliente Google WiFi.

Cor: branco sólido, sem luz, branco sólido
O que significa: O dispositivo está inicializando.
O que fazer:
Aguarde o dispositivo inicializar. Isso leva cerca de um minuto. Quando terminar, ele pulsará lentamente em branco, informando que está pronto para a configuração.

Cor: Branco de pulsação lenta
O que significa: O dispositivo está pronto para configuração.
O que fazer:
Use o app Google Home para configurar seu roteador.

Cor: Branco sólido
O que significa: o roteador está online e está tudo bem.
O que fazer:
Você está online. Aproveitar!

Cor: amarelo pulsando lentamente
O que significa: há um erro de rede.
O que fazer:
Verifique se o cabo Ethernet está conectado ao roteador e ao modem e se ambos os dispositivos estão ligados. Pode ser necessário desconectar e conectar cada dispositivo novamente.

Cor: Amarelo piscando rápido
O que significa: você está pressionando o botão de redefinição e redefinindo a configuração original deste dispositivo.
O que fazer:
Se você continuar pressionando o botão de reinicialização, após cerca de 12 segundos, a luz ficará amarela sólida. Quando estiver amarelo sólido, solte o botão de redefinição de fábrica.

Cor: amarelo sólido
O que significa: o roteador está sendo redefinido para as configurações de fábrica.
O que fazer:
Isso pode levar até 10 minutos. Quando terminar, o dispositivo será reinicializado e começará a piscar em branco, informando que está pronto para a configuração.
Imagem Luz vermelha constante Vermelho sólido Algo está errado. Falha crítica. Reinicie o roteador de fábrica. Se a luz permanecer vermelha, entre em contato com o suporte ao cliente WiFi.
"""

print(
    generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8).text
)

### Responder a perguntas como um personagem

Agora, você instrui o LLM a fingir ser Klara, uma pessoa. Você também contará ao modelo sobre os traços de personalidade de Klara. Em seguida, você pedirá ao modelo que responda a uma pergunta como Klara responderia.

In [ ]:
prompt = """
Você é a Klara.
Klara é gestora de investimentos.
Klara só responde se Klara tiver certeza de que está correto.
Klara responde à pergunta do usuário com base nos resumos das páginas abaixo.
Klara gera a ID de referência em que Klara encontrou a resposta para cada frase da resposta.
Klara também resume a parte onde se encontram as informações.

Resumos das páginas: ['ID de referência 1. O UBS quer ser visto como um banco global com raízes suíças, não apenas um banco europeu. O novo presidente do conselho, \
                    Colm Kelleher, e o CEO, Ralph Hamers, realizaram uma série de reuniões com influentes gestores de fundos dos EUA para aumentar suas participações \
                    no banco. O UBS é um dos bancos europeus mais valiosos com uma relação preço/valor contábil de 1.', 'ID de referência 2. Credit Suisse vende \
                    participação de 30% na gestora de ativos suíça Energy Infrastructure Partners. A EIP é especializada em investimentos de capital de longo prazo \
                    para ativos de infraestrutura de energia e renováveis em larga escala. A transação pode fazer parte de um esforço contínuo do banco suíço para \
                    reduzir ainda mais a lacuna de capital.' O príncipe Mohammed injetaria cerca de US$ 500 milhões na cisão do CS First Boston. Outros investidores \
                    podem incluir o ex-chefe do Barclays, Bob Diamonds, Atlas Merchant Capital. O Saudi National Bank já possui uma participação de 9,9 por cento na \
                    problemática instituição suíça. . O Wall Street Journal informou que a Apollo também está interessada em investir no CS First Boston. A divisão \
                    de banco de investimento de banco de investimento está prestes a ser desmembrada em uma nova unidade .', 'ID de referência 5. O banco privado \
                    Pictet, com sede em Genebra, contratou uma nova equipe em seu negócio de investimento no mercado privado de bilhões de dólares . Edmund Buckley, \
                    Nikolaus Hubmann, Sean Howard, Jan Dreesen e Hugo Hickson trabalharão na área de investimentos diretos em private equity no futuro. O \
                    comprometimento da equipe de Buckley pode ser visto como um golpe para Elif Aktuğ, parceiro da Pictet .', 'ID de referência 6. O UBS quer ser visto \
                    como um banco global com raízes suíças, não apenas um banco europeu . O novo presidente do conselho, Colm Kelleher, e o CEO, Ralph Hamers, \
                    realizaram uma série de reuniões com influentes gestores de fundos dos EUA para aumentar suas participações no banco. O UBS é um dos bancos \
                    europeus mais valiosos com uma relação preço/valor contábil de 1 .', 'ID de referência 7. A Apollo Global Management, sediada nos EUA, está \
                    entre um grupo de empresas financeiras em negociações com o Credit Suisse sobre uma participação no banco de investimento renovado. O Wall \
                    Street Journal informou que a Apollo também está interessada em investir no CS First Boston. A divisão de banco de investimento de bancos de \
                    investimento está definida para ser dividida em uma nova unidade.', 'ID de referência 8. A subsidiária da Apex com sede em Luxemburgo European \
                    Depositary Bank (EDB) e o especialista em securitização Gentwo firmam parceria . A EDB e a Gentwo oferecerão agentes pagadores e serviços \
                    bancários a investidores terceirizados em todo o mundo. Gentwo desenvolve plataformas para gerentes de ativos, bancos, escritórios familiares e \
                    capitalistas de risco .', 'Referência ID 9. O SNB tem perseguido planos para adquirir participações em instituições financeiras europeias e \
                    americanas há algum tempo . Credit Suisse, Julius Baer, Standard Chartered e o Asian DBS Group também foram mencionados como possíveis alvos .', \
                    'Referência ID 10. Zuercher Kantonalbank pretende expandir seus negócios de private banking, inclusive no exterior. O CEO Urs Baumann vê o Credit \
                    Suisse como um parceiro confiável para o banco estatal. A Baumann continuará a se concentrar em reduzir a dependência dos bancos no negócio de \
                    diferencial de juros em uma tentativa de diversificar seus negócios.']

Pergunta do usuário: Os bancos globais estão investindo em iniciativas de sustentabilidade?
Resposta da Klara:
"""

print(
    generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=40, top_p=0.8).text
)

### Convertendo uma lista de ingredientes para o formato JSON

Suponha que você queira listar ingredientes em receitas para inserir em um banco de dados, o que requer uma saída bem formatada como JSON. Isso pode ser feito usando um LLM da seguinte maneira:

In [ ]:
prompt = """
Extraia os ingredientes da seguinte receita. Retorne, em português, os ingredientes no formato JSON com as chaves: ingrediente, quantidade, tipo.

Ingredientes:
* 1 colher de sopa de azeite
* 1 cebola picada
* 2 cenouras picadas
* 2 talos de aipo picados
* 1 colher de chá de cominho moído
* 1/2 colher de chá de coentro moído
* 1/4 colher de chá de açafrão em pó
* 1/4 colher de chá de pimenta caiena (opcional)
* Sal e pimenta a gosto
* 425g de feijão preto, lavado e escorrido
* 425g de feijão, lavado e escorrido
* 420g lata de tomate em cubos, não escorrido
* 280g lata de tomate em cubos com pimentões verdes, não drenados
* 4 xícaras de caldo de legumes
* 1 xícara de coentro fresco picado
"""

print(
    generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text
)

### Organizando os resultados de uma extração de informações

Nesta seção, você extrai as informações desejadas de um bloco de texto e as organiza de maneira estruturada, como separar por vírgulas. Aqui você usa prompt few-shot para orientar o modelo a formatar suas saídas para serem separadas por vírgulas.

In [ ]:
prompt = """
Mensagem: Rachel Green (Jennifer Aniston), uma mulher protegida, mas amigável, foge do dia do casamento e de uma vida rica, mas insatisfatória, e encontra a amiga de infância Monica Geller (Courteney Cox), uma chef tensa, mas atenciosa.
Rachel se torna garçonete no café Central Perk do West Village depois que ela se muda para o apartamento de Monica acima do Central Perk e se junta ao grupo de amigas solteiras de Monica na casa dos 20 anos:
a colega de quarto anterior Phoebe Buffay (Lisa Kudrow), uma estranha massagista e musicista; o vizinho Joey Tribbiani (Matt LeBlanc), um ator obstinado, mas leal e esforçado; O colega de quarto de Joey, Chandler Bing ( Matthew Perry ),
um processador de dados sarcástico e autodepreciativo; e o irmão mais velho de Monica e colega de quarto de faculdade de Chandler, Ross Geller (David Schwimmer), um paleontólogo doce, mas inseguro.

Extraia os personagens e os atores que os interpretaram da mensagem acima:
Rachel Green - Jennifer Aniston, Monica Geller - Courteney Cox, Phoebe Buffay - Lisa Kudrow, Joey Tribbiani - Matt LeBlanc, Chandler Bing - Matthew Perry, Ross Geller - David Schwimmer

Mensagem: Jogos como xadrez, pôquer, Go e muitos videogames sempre foram um terreno fértil para a pesquisa de IA. Diplomacia é um jogo de negociação e formação de alianças para sete jogadores, jogado em um antigo mapa da Europa dividido
em províncias, onde cada jogador controla várias unidades (regras de Diplomacia). Na versão padrão do jogo, chamada Press Diplomacy, cada turno inclui uma fase de negociação, após a qual todos os jogadores revelam seu
movimentos escolhidos simultaneamente. O coração da Diplomacia é a fase de negociação, onde os jogadores tentam chegar a um acordo sobre seus próximos movimentos. Por exemplo, uma unidade pode apoiar outra unidade, permitindo-lhe superar a resistência de outras unidades,
conforme ilustrado aqui: Abordagens computacionais para Diplomacia têm sido pesquisadas desde a década de 1980, muitas das quais foram exploradas em uma versão mais simples do jogo chamado No-Press Diplomacy, onde a comunicação estratégica entre
jogadores não é permitido. Os pesquisadores também propuseram protocolos de negociação amigáveis ao computador, às vezes chamados \342\200\234Restricted-Press\342\200\235.

Extraia a definição de Diplomacia:
Um jogo de negociação e formação de alianças para sete jogadores


Mensagem: Em 2016, quando não estávamos usando simulação e estávamos usando uma pequena configuração de laboratório de robôs industriais para aprender a pegar pequenos objetos como brinquedos, chaves e utensílios domésticos do dia a dia, levou o equivalente a
quatro meses para um robô aprender a realizar uma pegada simples com uma taxa de sucesso de 75%. Hoje, um único robô aprende a realizar uma tarefa complexa como abrir portas com uma taxa de sucesso de 90%% em menos de um dia
de aprendizado do mundo real. Ainda mais empolgante, mostramos que podemos desenvolver os algoritmos e aprendizados da abertura de portas e aplicá-los a uma nova tarefa: endireitar cadeiras em nossos cafés. Este progresso nos dá
esperamos que nossa tentativa de construir robôs de aprendizagem de uso geral seja possível.

Extraia as taxas de sucesso dos robôs em 2016 e hoje, respectivamente:
75%, 90%

Mensagem: A CapitalG foi fundada há uma década para capacitar empreendedores com a experiência incomparável da Alphabet e do Google em crescimento.
Temos o privilégio de compartilhar as lições aprendidas ao ajudar a escalar Google, Stripe, Airbnb, CrowdStrike, Databricks e Zscaler com a próxima onda de empresas de tecnologia geracional, talvez incluindo a sua.
A Alphabet é nosso único LP e fornece capital paciente e de longo prazo. Como um fundo de crescimento independente, nossas prioridades estão alinhadas com nossos empreendedores. As empresas CapitalG atingiram o ajuste do mercado de produtos e estão prontas para escalar. Mantemos um portfólio pequeno e concentrado para que cada empresa receba capital substancial e suporte prático.

Extraia as empresas financiadas pela CapitalG:
"""

print(
    generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8).text
)

Como você pode ver na saída acima, com base no prompt few-shot, o LLM gerou a lista de nomes das empresas financiadas pelo CapitalG no formato separado por vírgulas.